# Eye Direction Labeling
The following note book is a machine learing CNN model using pytorch to label the direction of eyes in images. It uses the open source data from the eye-dataset on kaggle posted by kayvanshah. https://www.kaggle.com/datasets/kayvanshah/eye-dataset

The original intent for the dataset was for automating an wheelchair, but we are repurposing it for future uses in eye tracking.

The data consists of four folder and comprising a total of 14.5k images with the labels
- close_look : images of closed eye,
- left_look : images of eye looking at left,
- right_look : images of eye looking at right,
- forward_look : images of eye looking straight.



## Install the necessary python libraries
This is a step needed if you don't have these libraries or if running the code for the first time in colab.

In [ ]:
#Installs (do as needed)
!pip install torch torchvision
!pip install opendatasets
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## Import all necessary python libraries


In [ ]:
#Imports
import opendatasets as od
import torch
from torchvision import datasets, transforms
from torch.utils import data
import os
import cv2
import matplotlib.pyplot as plt
import torchvision
from torcheval.metrics import MulticlassAccuracy
import torcheval
import numpy as np
import random
from google.colab.patches import cv2_imshow                                     # needed to use imshow from OpenCV in colab

## Download data
Must input your own kaggle username and password.

In [ ]:
#Download the eye data set for direction of eyes (input your username and password for kaggle)
od.download("https://www.kaggle.com/datasets/kayvanshah/eye-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

## Get the dataset from the appropriate folder
Transform to a tensor and crop the image size to 50x50


In [ ]:
#get the dataset
img_size =50 #this value can be altered
dataset = datasets.ImageFolder(root = "eye-dataset/Eye dataset/", transform = transforms.Compose([
        transforms.RandomResizedCrop(img_size),  # image size int or tuple
        # Add more transforms here
        transforms.ToTensor(),  # convert to tensor at the end
]))

In [ ]:
print(dataset.class_to_idx) #this print out proves we are reading the code right to have the correct labels

{'close_look': 0, 'forward_look': 1, 'left_look': 2, 'right_look': 3}


## Prepare the data and form the train, test, and validation set

In [ ]:
# to generate random number list
shuffle_list = random.sample(range(0,len(dataset)), len(dataset))

#make the sizes for the different data set (sections)
train_size = 4*len(dataset)//5
test_size = int(0.1 * len(dataset) + train_size)
val_size  = int(0.1 * len(dataset) + test_size)

14360
14360


In [ ]:
#get the test and train data sets from random section of dataset

# val_set = torch.utils.data.Subset(dataset,[0, 4*len(dataset)/5])
train_set = torch.utils.data.Subset(dataset,shuffle_list[0:train_size]) #with labels
test_set = torch.utils.data.Subset(dataset,shuffle_list[train_size:test_size])
val_set = torch.utils.data.Subset(dataset,shuffle_list[test_size:val_size])

#load the data for the model
test_loader = data.DataLoader(test_set, batch_size = 100, shuffle=True)
train_loader = data.DataLoader(train_set, batch_size = 100, shuffle=True)

## Set the model specifications

In [ ]:
# Define the model
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 6, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(6, 16, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(6, 16, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Flatten(start_dim = 1),
    torch.nn.Linear(1296,120), #16 * 5 * 5, 120), #size of the output after flatten
    torch.nn.ReLU(),
    torch.nn.Linear(120,84),
    torch.nn.ReLU(),
    torch.nn.Linear(84,4), #total number pixels after flatten, number class
)

Define the optimizer, loss function, and epoch size for the model

In [ ]:
# Define the optimizer and loss function and epoch size
optimizer = torch.optim.Adam(model.parameters(), lr=0.001 )
loss_fn = torch.nn.CrossEntropyLoss()
epoch = 100

## Load the model IF ALREADY RAN AND SAVED THE MODEL


In [ ]:
# Load the training data
train_data = torch.load('train_data.pt') #from previously trained model

## Train the model and evaluate at each epoch

In [ ]:
#initialize the counter and the list of losses and accuracy
i=0
losses = []
accuracy = []

# Train the model
for epoch in range(100):
    correct = 0
    for images, labels in train_loader: #image size = [batch_size, channels, height, width]
        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #save the loss values
        losses.append(loss.item())

        #get the accuracy
        metric = MulticlassAccuracy()
        metric.update(outputs, labels)
        acc = metric.compute()
        accuracy.append(acc)

        #get the confusion matrix, f score, recall, precision
        conf_matrix = torcheval.metrics.functional.multiclass_confusion_matrix(outputs, labels, 4)
        f_score = torcheval.metrics.functional.multiclass_f1_score(outputs, labels, num_classes = 4, average = 'macro')
        recall = torcheval.metrics.functional.multiclass_recall(outputs, labels, num_classes = 4, average = 'macro')
        precision = torcheval.metrics.functional.multiclass_precision(outputs, labels, num_classes = 4, average = 'macro')

        # Print progress
        # if i % 100 == 0:
          # print(f'Epoch: {epoch + 1}/{100}, Batch: {i}, Loss: {loss.item():.4f}, accuracy {acc}, confMatrix {conf_matrix}')

        #update i
        i = i + 1
    print(f'Epoch: {epoch + 1}/{100}, Batch: {i}, Loss: {loss.item():.4f}, accuracy {acc}, confMatrix {conf_matrix}, fScore {f_score}, recall {recall}, precision {precision}')

Epoch: 1/100, Batch: 115, Loss: 1.0357, accuracy 0.5227272510528564, confMatrix tensor([[24,  0,  0,  0],
        [ 7,  0,  4,  7],
        [ 4,  1, 12,  4],
        [ 8,  0,  7, 10]]), fScore 0.4241637587547302, recall 0.4928571283817291, precision 0.3890172839164734
Epoch: 2/100, Batch: 230, Loss: 0.9558, accuracy 0.5568181872367859, confMatrix tensor([[21,  1,  1,  0],
        [ 9,  6,  6,  3],
        [ 4,  2, 14,  2],
        [ 5,  3,  3,  8]]), fScore 0.5298620462417603, recall 0.5551149249076843, precision 0.5592948794364929
Epoch: 3/100, Batch: 345, Loss: 1.1145, accuracy 0.4886363744735718, confMatrix tensor([[15,  5,  4,  1],
        [ 3,  8,  1,  4],
        [ 4, 11,  7,  1],
        [ 2,  4,  5, 13]]), fScore 0.482633113861084, recall 0.4865036606788635, precision 0.5016723871231079
Epoch: 4/100, Batch: 460, Loss: 0.8073, accuracy 0.6931818127632141, confMatrix tensor([[24,  1,  1,  1],
        [ 6,  8,  2,  5],
        [ 1,  1, 15,  3],
        [ 2,  2,  2, 14]]), fScore 0

## Evaluate the model
The resulting loss, accuracy, and confusion matrix at the final run

As described in the dataset previous models achieved a hight accuracy of The images in the dataset are regularized and augmented so that we can achieve more than 96% accuracy on the first training session.

In [ ]:
#print the loss, accuracy, confusion matrix, precision, recall, and the F1-score of the final epoch
print(f'Loss: {loss.item():.4f}, accuracy {acc}, confMatrix {conf_matrix}, fScore {f_score}, recall {recall}, precision {precision}')

Loss: 0.3417, accuracy 0.8863636255264282, confMatrix tensor([[22,  0,  1,  0],
        [ 0, 19,  1,  2],
        [ 1,  0, 16,  2],
        [ 0,  3,  0, 21]]), fScore 0.8855414390563965, recall 0.8843158483505249, precision 0.8872617483139038


plot the loss and accuracy

In [ ]:
plt.title("Loss Function and Accuracy")
plt.ylabel("Loss & and Accuracy")
plt.xlabel("Images")
plt.plot(losses)
plt.plot(accuracy)
plt.show()

NameError: name 'plt' is not defined

## Save the trained model to be used for testing and validation


In [ ]:
# Save the trained model
torch.save(model, 'trained_model.pt')
print("saved model!")

saved model!


##Load the model for testing

In [ ]:
#load the model
model_trained = torch.load('trained_model.pt')
print("Loaded model!")

Loaded model!


## Test the model

In [ ]:
#test the model with the test set
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for images, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = model_trained(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 88 %


## Show the images with the predictive labels

In [ ]:
#show the testing set with the predictions (set 20)

# dataiter = iter(test_loader)
# images, labels = next(dataiter)

#what is the len of images and labels
#set up for the text labels on the images

# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (5, 5)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

labels_str = ('close_look', 'forward_look', 'left_look', 'right_look')

count = 0
for images, labels in test_loader:
  labels_list = labels.flatten().tolist()

  for img in images:
    torchvision.utils.save_image(img, 'image%d.png' %count)

    #read back the image
    img = cv2.imread('image%d.png' %count)

    #put the label on the image
    img = cv2.putText(img, labels_str[labels_list[count]], org, font,
                   fontScale, color, thickness, cv2.LINE_AA)
    #show the image
    cv2_imshow(img)
    print(labels_str[labels_list[count]])

    count += 1
    if count == 20:
      break



left_look


close_look


left_look


right_look


right_look


...

forward_look


left_look


forward_look


right_look


IndexError: list index out of range